# 프로젝트: 퍼스널 컬러 진단 - VGG16

## 목차
1. ImageNet으로 사전 학습된 VGG16  
참고 링크: https://han-py.tistory.com/213  
2. Custom VGG16 (미완성)
3. VGG16에 퍼스널컬러(웜/쿨) 적용하기
참고 링크:https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=dnjswns2280&logNo=221738517884  

# 1. ImageNet으로 사전 학습된 VGG16

In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import glob
from PIL import Image

In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array # 오류 발생시 tensorflow 재설치하고 런타임 재연결 추천
from tensorflow.keras.applications import vgg16
from IPython.display import display # 이미지 출력 함수

In [6]:
# VGG16 모델 불러오기
model = vgg16.VGG16()

# 모델의 모양을 보여준다.
model.summary()

553467096/553467096 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [14]:
def predict_vgg16(model, filename) :

  # 이미지 파일을 읽고 화면에 표시
  image = load_img(filename)
  # image = PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=688x550
  display(image)


  # 모델 사이즈로 이미지 파일을 읽기
  image = load_img(filename, target_size=(224, 224))
  # image = PIL.Image.Image image mode=RGB size=224x224


  # 이미지 데이터를 numpy로 변환
  image = img_to_array(image)
  # image.shape = (224, 224, 3)

  # vgg16.preprocess_input()을 호출하기 위해 차원을 조정
  # 보통 모델을 여러 이미지를 한번에 호출.
  # 맨 앞의 1 : 이미지 갯수가 1개라는 것.
  # 두번째 224 : 가로
  # 세번째 224 : 세로
  # 네번째 3 : R, G, B 3개
  image = image.reshape((1, 224, 224, 3))

  # VGG16 모델 호출을 위해 데이터 전처리.
  # -255 ~ 255 사이 값으로 정규화한다.
  # 그리고 RGB를 BGR순으로 바꾼다.
  image = vgg16.preprocess_input(image)


  # 이미지를 모델에 적용
  yhat = model.predict(image)
  # yhat = [[2.03485320e-06 4.21382174e-06 1.45730738e-07 1.04057870e-06
  #          6.61934010e-08 2.63145339e-04 4.49358195e-05 2.03222541e-08
  #          ... ]] # 1000개 클래스에 대한 결과값.
  #


  # 모델 적용된 결과를 파싱
  label = vgg16.decode_predictions(yhat)
  # label = [[('n02655020', 'puffer', 0.9612253), ... ]]

  # 가장 확률이 높은 결과를 획득
  label = label[0][0]
  # label = ('n02655020', 'puffer', 0.9612253)

  # 라벨과 라벨을 예측한 확률을 출력
  print('%s (%.2f%%)' % (label[1], label[2]*100))

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
# 사진 경로
files = [
    "/content/drive/MyDrive/four_seasons/데이터셋_jpg/고양이/고양이.jpg",
    "/content/drive/MyDrive/four_seasons/데이터셋_jpg/고양이/고양이2.jpg",
    "/content/drive/MyDrive/four_seasons/데이터셋_jpg/웜톤/웜톤_jpg/리사.jpg",
    "/content/drive/MyDrive/four_seasons/데이터셋_jpg/쿨톤/쿨톤_jpg/김고은.jpg"
]

In [26]:
# 각 사진마다 예측된 라벨과 예측한 확률 확인하기
for file in files :
  predict_vgg16(model, file)

'/content/drive/MyDrive/four_seasons/데이터셋_jpg/고양이/고양이.jpg'

1/1 [==============================] - 1s 561ms/step
Egyptian_cat (48.11%)


'/content/drive/MyDrive/four_seasons/데이터셋_jpg/고양이/고양이2.jpg'

1/1 [==============================] - 1s 585ms/step
tabby (37.17%)


'/content/drive/MyDrive/four_seasons/데이터셋_jpg/웜톤/웜톤_jpg/리사.jpg'

1/1 [==============================] - 1s 536ms/step
sweatshirt (38.55%)


'/content/drive/MyDrive/four_seasons/데이터셋_jpg/쿨톤/쿨톤_jpg/김고은.jpg'

1/1 [==============================] - 1s 541ms/step
pajama (15.46%)


# 2. Custom VGG16
- 미완성. fit부분이 맞지 않음

In [28]:
import keras
def build_vgg_block(input_layer,
                    num_cnn=3,
                    channel=64,
                    block_num=1,
                   ):
    # 입력 레이어
    x = input_layer
	# num_cnn : 한블럭에서 사용할 conv필터 개수 네트워크에 따라 2개일때가 있고 3개일때가 있음.
    # CNN 레이어
    for cnn_num in range(num_cnn):
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_conv{cnn_num}'
        )(x)

    # Max Pooling 레이어
    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=2,
        name=f'block{block_num}_pooling'
    )(x)

    return x

In [30]:
def build_vgg(input_shape=(32,32,3),
              num_cnn_list=[2,2,3,3,3],
              channel_list=[64,128,256,512,512],
              num_classes=10):

    assert len(num_cnn_list) == len(channel_list) #모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.




    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    output = input_layer

    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_vgg_block(
            output,
            num_cnn=num_cnn,
            channel=channel,
            block_num=i
        )

    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(4096, activation='relu', name='fc1')(output)
    output = keras.layers.Dense(4096, activation='relu', name='fc2')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)

    model = keras.Model(
        inputs=input_layer,
        outputs=output
    )
    return model

In [ ]:
# vgg_19 = build_vgg(
#     num_cnn_list=[2,2,4,4,4],
#     channel_list=[64,128,256,512,512]
# )

In [33]:
vgg_16 = build_vgg()

In [34]:
BATCH_SIZE = 32
EPOCH = 20

In [40]:
import tensorflow as tf

# CIFAR-10 데이터셋 로드
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 데이터 정규화 (0~1 사이 값으로 스케일링)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 레이블을 One-Hot 인코딩
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


170498071/170498071 [==============================] - 2s 0us/step


In [43]:
vgg_16.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

# history_16 = vgg_16.fit(
#     x_train,
#     epochs=EPOCH,
#     validation_data=x_test,
#     verbose=1,
#     use_multiprocessing=True,
# )
history_16 = vgg_16.fit(
    x_train,
    y_train,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    validation_split=0.1
)

Epoch 1/20


InvalidArgumentError: ignored

# 3. VGG16에 퍼스널컬러(웜/쿨) 적용하기

## (1) 라이브러리 및 데이터 불러오기

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import vgg16

import numpy as np
import cv2
import os
import glob
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 파일 불러오는 함수
def load_images_from_directory(directory_path):
    image_paths = glob.glob(directory_path + '/*.jpg')  # 디렉토리 내의 모든 jpg 파일 경로 찾기
    image_paths.extend(glob.glob(directory_path + '/*.jpeg'))  # 디렉토리 내의 모든 jpeg 파일 경로 추가

    images = []
    for image_path in image_paths:
        image = Image.open(image_path)
        image_np = np.array(image)
        images.append(image_np)

    return images

In [34]:
# 모든 이미지 불러오기
directory_path = "/content/drive/MyDrive/four_seasons/사계절_연예인 이미지 데이터셋/prototype/웜톤_jpg"  # 디렉토리 경로
directory_path2 = "/content/drive/MyDrive/four_seasons/사계절_연예인 이미지 데이터셋/prototype/쿨톤_jpg"  # 디렉토리 경로
images = load_images_from_directory(directory_path)
images2 = load_images_from_directory(directory_path2)

length = len(images)
print(length)
print(len(images2))

# 이미지 리스트 합치기
images = np.concatenate((images, images2), axis=0)
print(len(images))

50
50
100


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


## (2) 데이터 전처리

In [35]:
def preprocess_image(image):
    # 이미지 크기 조절
    image = tf.image.resize(image, (224, 224))
    # 이미지를 [0, 1] 범위로 정규화
    image = image / 255.0
    return image

images = np.array([preprocess_image(image) for image in images])
print(images.shape)

(100, 224, 224, 3)


In [36]:
# labels에 대한 코드
num = len(images)
labels = np.zeros(num)

# 뒤쪽의 50개의 요소를 1로 변경
labels[length:] = 1

# 0이 웜톤 1이 쿨톤
print(len(images))
print(labels.shape)
print(labels)

100
(100,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


## (3) train, val 분리하기

In [37]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(images,
                                                  labels,
                                                  test_size=0.3,
                                                  random_state=1024)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

x_train:  (70, 224, 224, 3)
y_train:  (70,)
x_val:  (30, 224, 224, 3)
y_val:  (30,)


In [38]:
# 데이터를 텐서로 변환
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.int32)

## (4) 모델 정의 및 컴파일

In [42]:
# VGG16 모델 불러오기
vgg16_model = vgg16.VGG16()

# 추가하기
model = models.Sequential()
model.add(vgg16_model)
model.add(layers.Dense(2, activation='softmax'))

In [43]:
# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


## (5) 모델 학습하기

In [44]:
# 7분걸림
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=2,
                    validation_data=(x_val, y_val))

Epoch 1/2
2/2 [==============================] - 207s 39s/step - loss: 0.6929 - accuracy: 0.5000 - val_loss: 0.7007 - val_accuracy: 0.4000
Epoch 2/2
2/2 [==============================] - 216s 59s/step - loss: 0.6908 - accuracy: 0.5429 - val_loss: 0.7010 - val_accuracy: 0.4000


## (6) 예측하기

In [45]:
pred = model.predict(x_val)
pred_class = np.argmax(pred, axis=1)
y_val = np.array(y_val)

1/1 [==============================] - 20s 20s/step


In [46]:
print(pred_class)
print(y_val)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0]


In [47]:
acc = np.mean(pred_class == y_val)
print('accuracy: %f' % (acc,))

accuracy: 0.400000


## (7) TEST

In [48]:
class_name = ["웜톤", "쿨톤"]

In [49]:
# test 불러오기 (낱개 버전)
test_path = "/content/drive/MyDrive/four_seasons/사계절_연예인 이미지 데이터셋/prototype/연예인 사진 10장/01.jpg"
x_test = Image.open(test_path)
x_test = np.array(x_test)
print(x_test.shape)

# 배열의 크기 변환
x_test = x_test.reshape((1, x_test.shape[0], x_test.shape[1], 3))
print("변환된 배열 크기:", x_test.shape)

# 전처리(resize, 정규화)
x_test = preprocess_image(x_test)
print(x_test.shape)

# 만약 앱구현과 연결된다면 저장된 모델을 다시 돌릴 필요없이 저장된 최적의 모델을 사용하도록 수정해야함
# test
pred = model.predict(x_test)
pred_class = np.argmax(pred, axis=1)
print("당신은", class_name[int(pred_class)], "입니다~!")
print("웜톤일 확률: ", np.round(pred[:, 0], 3))
print("쿨톤일 확률: ", np.round(pred[:, 1], 3))

(600, 600, 3)
변환된 배열 크기: (1, 600, 600, 3)
(1, 224, 224, 3)
1/1 [==============================] - 1s 601ms/step
당신은 쿨톤 입니다~!
웜톤일 확률:  [0.482]
쿨톤일 확률:  [0.518]


In [ ]:
# 아이유는 웜톤이다ㅠ